In [ ]:
from Xray.pipeline.training_pipeline import TrainPipeline
from Xray.entity.artifact_entity import DataIngestionArtifact,DataTransformationArtifact
from Xray.entity.config_entity import ModelEvaluationConfig
from Xray.components.model_evaluation import ModelEvaluation
from torch.utils.data import DataLoader
# data_artifact = DataTransformationArtifact()

# pipe = TrainPipeline()



In [ ]:
train_path = r"D:\Projects\deeplearningproject\artifact\11_05_2024_23_13_14\data_transformation\train_transforms.pkl"
test_path = r"D:\Projects\deeplearningproject\artifact\11_05_2024_23_13_14\data_transformation\test_transforms.pkl"
model_path = r"artifact/11_05_2024_23_13_14/trained_model/model.pt"
train_data=DataLoader(train_path)
test_data = DataLoader(test_path)
data_artifact = DataTransformationArtifact(test_transform_file_path=test_path,
                                           train_transform_file_path=train_path,
                                           transformed_test_object=test_data,
                                           transformed_train_object=train_data)

config = ModelEvaluationConfig()

In [36]:
model_eval =ModelEvaluation(data_transformation_artifact=data_artifact,
                            model_evaluation_config=config,
                            model_training_artifact=model_path)
model_eval.initiate_model_evaluation()

ModelEvaluationArtifact(model_accuracy=None)

In [2]:
artifact.test_file_path

'D:\\Projects\\deeplearningproject\\artifact\\11_05_2024_08_32_32\\data_ingestion\\data\\test'

In [ ]:
data_artifact = pipe.start_model_trainer(data_ingestion_artifact=artifact)

In [16]:
import os
os.path.exists(artifact.train_file_path)

True

In [4]:
data_artifact.test_transform_file_path

'artifact\\11_05_2024_08_55_10\\data_transformation\\test_transforms.pkl'